In [1]:
# Example 7-4: 이미지 분류
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

import os
import numpy as np
import pandas as pd
from PIL import Image
from sklearn import datasets, model_selection

In [2]:
dirs = ['ants', 'bees']
data = []
label = []

for i, d in enumerate(dirs):
    files = os.listdir('../data/hymenoptera_data/' + d)
    for f in files:
        img = Image.open('../data/hymenoptera_data/' + d + '/' + f, 'r')
        resize_img = img.resize((128,128))
        r,g,b = resize_img.split()
        r_resize_img = np.asarray(np.float32(r)/255.0)
        g_resize_img = np.asarray(np.float32(g)/255.0)
        b_resize_img = np.asarray(np.float32(b)/255.0)
        rgb_resize_img = np.asarray([r_resize_img, g_resize_img, b_resize_img])
        data.append(rgb_resize_img)
        label.append(i)

In [3]:
pd.DataFrame(data[0][0])

,0,1,2,3,4,5,6,7,8,9,...,118,119,120,121,122,123,124,125,126,127
0,0.443137,0.396078,0.396078,0.333333,0.329412,0.380392,0.380392,0.360784,0.388235,0.466667,...,0.384314,0.372549,0.360784,0.368627,0.364706,0.337255,0.376471,0.294118,0.313726,0.411765
1,0.415686,0.360784,0.364706,0.372549,0.380392,0.392157,0.400000,0.333333,0.415686,0.509804,...,0.419608,0.372549,0.403922,0.384314,0.400000,0.301961,0.290196,0.250980,0.321569,0.415686
2,0.396078,0.415686,0.345098,0.380392,0.396078,0.419608,0.376471,0.380392,0.450980,0.478431,...,0.337255,0.396078,0.403922,0.372549,0.345098,0.341176,0.243137,0.282353,0.368627,0.403922
3,0.450980,0.341176,0.372549,0.360784,0.392157,0.384314,0.388235,0.329412,0.447059,0.486275,...,0.352941,0.384314,0.450980,0.423529,0.345098,0.356863,0.258824,0.388235,0.462745,0.427451
4,0.443137,0.372549,0.352941,0.356863,0.356863,0.364706,0.364706,0.337255,0.498039,0.443137,...,0.403922,0.380392,0.392157,0.313726,0.298039,0.400000,0.380392,0.419608,0.454902,0.419608
5,0.329412,0.286275,0.345098,0.317647,0.360784,0.325490,0.349020,0.380392,0.478431,0.490196,...,0.321569,0.227451,0.321569,0.341176,0.266667,0.278431,0.345098,0.403922,0.470588,0.462745
6,0.325490,0.294118,0.298039,0.301961,0.388235,0.317647,0.305882,0.517647,0.529412,0.505882,...,0.301961,0.254902,0.231373,0.274510,0.301961,0.435294,0.443137,0.529412,0.490196,0.478431
7,0.325490,0.290196,0.298039,0.290196,0.333333,0.290196,0.313726,0.517647,0.478431,0.501961,...,0.313726,0.321569,0.286275,0.262745,0.333333,0.458824,0.482353,0.505882,0.513726,0.494118
8,0.337255,0.345098,0.301961,0.286275,0.305882,0.278431,0.494118,0.486275,0.498039,0.498039,...,0.286275,0.313726,0.258824,0.309804,0.458824,0.466667,0.435294,0.462745,0.501961,0.474510
9,0.333333,0.321569,0.305882,0.349020,0.305882,0.384314,0.525490,0.525490,0.513726,0.498039,...,0.317647,0.290196,0.337255,0.431373,0.454902,0.435294,0.458824,0.454902,0.482353,0.443137


In [4]:
label

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,


In [5]:
data = np.array(data, dtype='float32')
label = np.array(label, dtype='int64')

In [6]:
train_X, test_X, train_Y, test_Y = model_selection.train_test_split(data, label, test_size=0.1)
print(len(train_X))
print(len(test_X))

357
40


In [7]:
train_X = torch.from_numpy(train_X).float()
train_Y = torch.from_numpy(train_Y).long()
print(train_X.shape)
print(train_Y.shape)

torch.Size([357, 3, 128, 128])
torch.Size([357])


In [8]:
train = TensorDataset(train_X, train_Y)
print(train[0])
train_loader = DataLoader(train, batch_size=32, shuffle=True)

(tensor([[[0.1961, 0.1804, 0.3020,  ..., 0.1686, 0.2784, 0.4000],
         [0.2235, 0.1608, 0.5882,  ..., 0.2314, 0.3529, 0.2706],
         [0.2627, 0.3294, 0.5333,  ..., 0.2980, 0.6941, 0.2510],
         ...,
         [0.7059, 0.8157, 0.7294,  ..., 0.8078, 0.8039, 0.8039],
         [0.8588, 0.7647, 0.8392,  ..., 0.7843, 0.7882, 0.7529],
         [0.7922, 0.7804, 0.8157,  ..., 0.8157, 0.8510, 0.7882]],

        [[0.2863, 0.1608, 0.4549,  ..., 0.2471, 0.4000, 0.3098],
         [0.3882, 0.2275, 0.6157,  ..., 0.2941, 0.4000, 0.2588],
         [0.4627, 0.3255, 0.5647,  ..., 0.4235, 0.6863, 0.2941],
         ...,
         [0.7059, 0.7961, 0.7255,  ..., 0.8000, 0.8039, 0.8000],
         [0.8588, 0.7569, 0.8157,  ..., 0.7647, 0.8078, 0.7490],
         [0.7804, 0.7569, 0.8039,  ..., 0.8196, 0.7725, 0.7843]],

        [[0.0745, 0.0431, 0.1451,  ..., 0.0549, 0.1647, 0.1373],
         [0.1412, 0.0235, 0.4431,  ..., 0.1020, 0.3059, 0.0588],
         [0.1686, 0.2039, 0.2627,  ..., 0.1255, 0.5961, 0

In [9]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3,10,5)
        self.conv2 = nn.Conv2d(10,20,5)
        self.fc1 = nn.Linear(20*29*29,50)
        self.fc2 = nn.Linear(50,2)
    def forward(self,x):
        x = F.max_pool2d(F.relu(self.conv1(x)),2)
        x = F.max_pool2d(F.relu(self.conv2(x)),2)
        x = x.view(-1,20*29*29)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x,dim=0)
    
model = Net()

In [10]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
for epoch in range(250):
    total_loss = 0
    for train_x, train_y in train_loader:
        optimizer.zero_grad()
        output = model(train_x)
        loss = criterion(output, train_y)
        loss.backward()
        optimizer.step()
        total_loss += loss.data.item()
    if (epoch+1)%50 == 0:
        print(epoch+1, total_loss)

50 0.01435555430362001
100 2.59181853197515
150 0.006809831873397343
200 0.004527075014266302
250 0.002519204899726901


In [11]:
#test_X = np.array(test_X, dtype='float32')
#test_Y = np.array(test_Y, dtype='int64')
test_X = torch.from_numpy(test_X).float()
test_Y = torch.from_numpy(test_Y).long()
print(test_X.shape)
print(test_Y.shape)

torch.Size([40, 3, 128, 128])
torch.Size([40])


In [12]:
result = torch.max(model(test_X).data, 1)[1]
accuracy = sum(test_Y.data.numpy()==result.numpy()) / len(test_Y.data.numpy())
accuracy

0.725